## Imports

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import minimize
from scipy.stats import bernoulli
from scipy.special import expit as sigmoid

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

## Load data

In [ ]:
TRAIN = pd.read_csv('data/preprocessed_train.csv')
VAL = pd.read_csv('data/preprocessed_val.csv')
TEST = pd.read_csv('data/preprocessed_test.csv')

## Implementation

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import cholesky, solve_triangular
from scipy.special import expit


def rbf_kernel(X1, X2, length_scale=1.0):
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return np.exp(-0.5 * sqdist / length_scale**2)


class GaussianProcessClassifier:
    def __init__(self, kernel=rbf_kernel, length_scale=1.0, noise=1e-6):
        self.kernel = kernel
        self.length_scale = length_scale
        self.noise = noise
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
        K = self.kernel(X, X, self.length_scale) + self.noise * np.eye(len(X))
        self.L_ = cholesky(K, lower=True)
        
        self.alpha_ = np.zeros_like(y, dtype=float)
        
        def objective(alpha):
            return 0.5 * np.dot(alpha.T, np.dot(K, alpha)) - np.sum(expit(y * alpha))
        
        def grad(alpha):
            return np.dot(K, alpha) - y * expit(-y * alpha)
        
        result = minimize(objective, self.alpha_, jac=grad, method='L-BFGS-B')
        self.alpha_ = result.x
    
    def predict_proba(self, X):
        K_trans = self.kernel(X, self.X_train, self.length_scale)
        f_star = np.dot(K_trans, self.alpha_)
        
        v = solve_triangular(self.L_, K_trans.T, lower=True)
        var_f_star = np.diag(self.kernel(X, X, self.length_scale)) - np.sum(v**2, axis=0)
        
        proba = expit(f_star / np.sqrt(1 + np.pi * var_f_star / 8))
        return proba
    
    def predict(self, X):
        return np.sign(self.predict_proba(X) - 0.5)



class MultiClassGaussianProcessClassifier:
    """
    Implements OVR classification using binary GP classifiers.
    """
    def __init__(self, kernel=rbf_kernel, length_scale=1.0, noise=1e-6):
        self.kernel = kernel
        self.length_scale = length_scale
        self.noise = noise
        self.classifiers = {}
    
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        for cls in self.classes_:
            y_binary = np.where(y == cls, 1, -1)
            gpc = GaussianProcessClassifier(kernel=self.kernel, length_scale=self.length_scale, noise=self.noise)
            gpc.fit(X, y_binary)
            self.classifiers[cls] = gpc

    
    def predict_proba(self, X):
        proba = np.zeros((X.shape[0], len(self.classes_)))
        for idx, cls in enumerate(self.classes_):
            proba[:, idx] = self.classifiers[cls].predict_proba(X)
        proba = np.clip(proba, 1e-10, 1-1e-10) 
        proba /= proba.sum(axis=1, keepdims=True) 
        return proba

    
    def predict(self, X):
        proba = self.predict_proba(X)
        return self.classes_[np.argmax(proba, axis=1)]

## Experiments

#### Accuracy sanity check

In [ ]:
X_train = TRAIN.drop(columns='y').to_numpy()
y_train = TRAIN.y.to_numpy()

X_test = VAL.drop(columns='y').to_numpy()
y_test = VAL.y.to_numpy()

multi_gpc = MultiClassGaussianProcessClassifier(length_scale=1.0, noise=1e-6)
multi_gpc.fit(X_train, y_train)

y_pred = multi_gpc.predict(X_test)

(y_pred == y_test).mean()

## Default params

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import label_binarize


def evaluate_model(X_train, y_train, X_test, y_test, num_runs=20):
    accuracies = []
    f1_scores = []
    precisions = []
    recalls = []
    aucs = []
    
    for _ in range(num_runs):
        multi_gpc = MultiClassGaussianProcessClassifier(length_scale=1.0, noise=1e-6)
        multi_gpc.fit(X_train, y_train)
        y_pred = multi_gpc.predict(X_test)
        
        accuracies.append(accuracy_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        
        y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
        y_pred_proba = multi_gpc.predict_proba(X_test)
        aucs.append(roc_auc_score(y_test_bin, y_pred_proba, average='weighted', multi_class='ovr'))
    
    print(f"Accuracy: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
    print(f"F1 Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
    print(f"Precision: {np.mean(precisions):.3f} ± {np.std(precisions):.3f}")
    print(f"Recall: {np.mean(recalls):.3f} ± {np.std(recalls):.3f}")
    print(f"AUC: {np.mean(aucs):.3f} ± {np.std(aucs):.3f}")
    
    fig, ax = plt.subplots(1, 5, figsize=(20, 4))
    metrics = [accuracies, f1_scores, precisions, recalls, aucs]
    titles = ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'AUC']
    
    for i in range(5):
        ax[i].boxplot(metrics[i])
        ax[i].set_title(titles[i])
        ax[i].set_xticks([1])
        ax[i].set_xticklabels([titles[i]])
    
    plt.tight_layout()
    plt.show()

## Scale param vs metrics (noise fixed)

In [ ]:
import itertools 

def fixed_noise(X_train, y_train, X_test, y_test, length_scales, noise=1e-6, num_runs=1):
    results = []

    for length_scale in length_scales:
        accuracies = []
        f1_scores = []
        precisions = []
        recalls = []
        aucs = []

        for _ in range(num_runs):
            multi_gpc = MultiClassGaussianProcessClassifier(length_scale=length_scale, noise=noise)
            multi_gpc.fit(X_train, y_train)
            y_pred = multi_gpc.predict(X_test)
            
            accuracies.append(accuracy_score(y_test, y_pred))
            f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
            precisions.append(precision_score(y_test, y_pred, average='weighted'))
            recalls.append(recall_score(y_test, y_pred, average='weighted'))
            
            y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
            y_pred_proba = multi_gpc.predict_proba(X_test)
            aucs.append(roc_auc_score(y_test_bin, y_pred_proba, average='weighted', multi_class='ovr'))
        
        results.append({
            'length_scale': length_scale,
            'accuracy_mean': np.mean(accuracies),
            'accuracy_std': np.std(accuracies),
            'f1_mean': np.mean(f1_scores),
            'f1_std': np.std(f1_scores),
            'precision_mean': np.mean(precisions),
            'precision_std': np.std(precisions),
            'recall_mean': np.mean(recalls),
            'recall_std': np.std(recalls),
            'auc_mean': np.mean(aucs),
            'auc_std': np.std(aucs)
        })
    
    return results

# Plotting function
def plot_results(results):
    fig, ax = plt.subplots(2, 3, figsize=(18, 12))
    metrics = ['accuracy', 'f1', 'precision', 'recall', 'auc']
    titles = ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'AUC']
    
    for i, metric in enumerate(metrics):
        metric_mean = [res[f'{metric}_mean'] for res in results]
        metric_std = [res[f'{metric}_std'] for res in results]
        
        ax[i // 3, i % 3].plot(range(len(results)), metric_mean, marker='o', label=f'{titles[i]} Mean')
        ax[i // 3, i % 3].fill_between(range(len(results)), 
                                        np.array(metric_mean) - np.array(metric_std), 
                                        np.array(metric_mean) + np.array(metric_std), 
                                        alpha=0.2, label=f'{titles[i]} Std')
        
        ax[i // 3, i % 3].set_title(titles[i])
        ax[i // 3, i % 3].set_xticks(range(len(results)))
        ax[i // 3, i % 3].set_xticklabels([f"LS: {res['length_scale']}" for res in results], rotation=45, ha="right")
        ax[i // 3, i % 3].legend()
    ax[1,2].set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [ ]:
length_scales = [0.1, 0.2, 0.5, 1.0, 5.0, 10.0]
fixed_noise_val = 1e-6

results = fixed_noise(X_train, y_train, X_test, y_test, length_scales, noise=fixed_noise_val)

In [ ]:
plot_results(results)

## Noise param vs metrics (scale fixed)

In [ ]:
def fixed_scale(X_train, y_train, X_test, y_test, noises, length_scale=1.0, num_runs=1):
    results = []

    for noise in noises:
        accuracies = []
        f1_scores = []
        precisions = []
        recalls = []
        aucs = []

        for _ in range(num_runs):
            multi_gpc = MultiClassGaussianProcessClassifier(length_scale=length_scale, noise=noise)
            multi_gpc.fit(X_train, y_train)
            y_pred = multi_gpc.predict(X_test)
            
            accuracies.append(accuracy_score(y_test, y_pred))
            f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
            precisions.append(precision_score(y_test, y_pred, average='weighted'))
            recalls.append(recall_score(y_test, y_pred, average='weighted'))
            
            y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
            y_pred_proba = multi_gpc.predict_proba(X_test)
            aucs.append(roc_auc_score(y_test_bin, y_pred_proba, average='weighted', multi_class='ovr'))
        
        results.append({
            'noise': noise,
            'accuracy_mean': np.mean(accuracies),
            'accuracy_std': np.std(accuracies),
            'f1_mean': np.mean(f1_scores),
            'f1_std': np.std(f1_scores),
            'precision_mean': np.mean(precisions),
            'precision_std': np.std(precisions),
            'recall_mean': np.mean(recalls),
            'recall_std': np.std(recalls),
            'auc_mean': np.mean(aucs),
            'auc_std': np.std(aucs)
        })
    return results

In [ ]:
length_scale = 1.0
noises = [1e-3,2e-3,3e-3, 4e-3,5e-3,6e-3,7e-3,8e-3, 1e-4, 5e-4,1e-5, 5e-5, 1e-6]
results = fixed_scale(X_train, y_train, X_test, y_test, noises, length_scale=length_scale)

In [ ]:
# Plotting function
def plot_nresults(results):
    fig, ax = plt.subplots(2, 3, figsize=(18, 12))
    metrics = ['accuracy', 'f1', 'precision', 'recall', 'auc']
    titles = ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'AUC']
    
    for i, metric in enumerate(metrics):
        metric_mean = [res[f'{metric}_mean'] for res in results]
        metric_std = [res[f'{metric}_std'] for res in results]
        
        ax[i // 3, i % 3].plot(range(len(results)), metric_mean, marker='o', label=f'{titles[i]} Mean')
        ax[i // 3, i % 3].fill_between(range(len(results)), 
                                        np.array(metric_mean) - np.array(metric_std), 
                                        np.array(metric_mean) + np.array(metric_std), 
                                        alpha=0.2, label=f'{titles[i]} Std')
        
        ax[i // 3, i % 3].set_title(titles[i])
        ax[i // 3, i % 3].set_xticks(range(len(results)))
        ax[i // 3, i % 3].set_xticklabels([f"noise: {res['noise']}" for res in results], rotation=45, ha="right")
        ax[i // 3, i % 3].legend()
    ax[1,2].set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_nresults(results)

## Hyperparam tuning with Optuna

In [ ]:
import optuna

In [ ]:
from sklearn.metrics import  f1_score

def objective(trial):
    param = {
        'length_scale': trial.suggest_float('length_scale', 0.1, 15),
        'noise': trial.suggest_float('noise', 1e-6, 1e-3)
    }
    multi_gpc = MultiClassGaussianProcessClassifier(length_scale=param['length_scale'], noise=param['noise'])
    multi_gpc.fit(X_train, y_train)
    y_pred = multi_gpc.predict(X_test) # to walidacyjny tak naprawde
    f1 = f1_score(y_test, y_pred, average='weighted')
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
best_trial = study.best_trial

In [ ]:
best_trial

In [ ]:
# Use only interactively, will be empty in Jupyter Lab due to optuna errors
def visualize_optuna(study):
    import optuna.visualization as vis
    
    optuna.visualization.plot_optimization_history(study).show()
    
    optuna.visualization.plot_param_importances(study).show()
    
    optuna.visualization.plot_slice(study).show()
    
    optuna.visualization.plot_parallel_coordinate(study).show()

In [ ]:
visualize_optuna(study)

## Final eval

In [ ]:
X_train = TRAIN.drop(columns='y').to_numpy()
y_train = TRAIN.y.to_numpy()
X_val = VAL.drop(columns='y').to_numpy()
y_val = VAL.y.to_numpy()

X_test = TEST.drop(columns='y').to_numpy()
y_test = TEST.y.to_numpy()

X_train_combined = np.concatenate((X_train, X_val), axis=0)
y_train_combined = np.concatenate((y_train, y_val), axis=0)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import label_binarize

def evaluate_final_model(X_train, y_train, X_test, y_test, num_runs=20):
    accuracies = []
    f1_scores = []
    precisions = []
    recalls = []
    aucs = []
    
    for _ in range(num_runs):
        multi_gpc = MultiClassGaussianProcessClassifier(length_scale=0.9370589034485247, noise=0.0007533926368261847)
        multi_gpc.fit(X_train, y_train)
        y_pred = multi_gpc.predict(X_test)
        
        accuracies.append(accuracy_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        
        y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
        y_pred_proba = multi_gpc.predict_proba(X_test)
        aucs.append(roc_auc_score(y_test_bin, y_pred_proba, average='weighted', multi_class='ovr'))
    
    print(f"Accuracy: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
    print(f"F1 Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
    print(f"Precision: {np.mean(precisions):.3f} ± {np.std(precisions):.3f}")
    print(f"Recall: {np.mean(recalls):.3f} ± {np.std(recalls):.3f}")
    print(f"AUC: {np.mean(aucs):.3f} ± {np.std(aucs):.3f}")
    
    fig, ax = plt.subplots(1, 5, figsize=(20, 4))
    metrics = [accuracies, f1_scores, precisions, recalls, aucs]
    titles = ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'AUC']
    
    for i in range(5):
        ax[i].boxplot(metrics[i])
        ax[i].set_title(titles[i])
        ax[i].set_xticks([1])
        ax[i].set_xticklabels([titles[i]])
    
    plt.tight_layout()
    plt.show()

In [ ]:
evaluate_final_model(X_train_combined, y_train_combined, X_test, y_test)